In [ ]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device('cuda:0')
    from torch.cuda import FloatTensor
else:
    device = torch.device('cpu')
    from torch import FloatTensor
    
try:
    from google.colab import drive
    is_in_colab = True
except:
    is_in_colab = False
    

In [ ]:
# вывод информации о выданном с colab GPU
if is_in_colab:
    !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
    !pip install gputil
    !pip install psutil
    !pip install humanize
    import psutil
    import humanize
    import os
    import GPUtil as GPU
    GPUs = GPU.getGPUs()
    gpu = GPUs[0]
    def printm():
        process = psutil.Process(os.getpid())
        print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
        print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

    printm()
    

In [ ]:
if is_in_colab:
    drive.mount('/content/drive')
    data_folder = r'/content/drive/My Drive/Colab/IDAO_2020/'
else:
    data_folder = r'./data/'

In [ ]:
# баш команда для создания каталога в монитрованном гугл-диске, для хранения там данных. 
# Выполните один раз после монтирования диска, чтобы не создавать папку вручную
# ! mkdir -p '/content/drive/My Drive/Colab/IDAO_2020/'


In [ ]:
def save_model(path, model, optimizer, scheduler, train_history, val_history):
    torch.save({
            'epoch': len(train_history),
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_history': train_history,
            'val_history': val_history
            }, path)
    print('successfully saved')
    
def load_model(path, model, optimizer, scheduler, train_history, val_history):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    train_history = checkpoint['train_history']
    val_history = checkpoint['val_history']
    print('successfully loaded')


In [ ]:
def add_delta_time(df, columns=None):
    """
    Добавляет столбец delta_time в секундах. Возвращает DataFrame в порядке указанном columns
    если columns нет то возвращает все столбцы
    """
    dataframe = df.sort_values(by=['sat_id', 'epoch'], inplace=False)
    dataframe['delta_time'] = dataframe.iloc[1:]['epoch'] - dataframe.iloc[0:-1]['epoch'].values
    dataframe['delta_seconds'] = dataframe['delta_time'].dt.seconds
    filters = dataframe['sat_id'] != np.insert(dataframe.iloc[0:-1]['sat_id'].values, 0, -1)
    dataframe.loc[filters, ['delta_time', 'delta_seconds']] = 0
    if not columns:
        columns=dataframe.columns
    return dataframe[columns]

In [ ]:
class Norm():
    """
    Нормализатор. 
    Init запоминает среднее и стандартное отклонение в данных
    """
    def __init__(self, df, ignore_column=None):
        self.mean = df.mean()
        self.std = df.std()
        self.l2 = None
        self._get_l2(df)
        if ignore_column:
            self.mean[ignore_column] = 0
            self.std[ignore_column] = 1
            self.l2[ignore_column] = 1
        
    def _get_l2(self, df):
        self.l2 = df.pow(2, axis=0).sum(axis=0).pow(0.5, axis=0) / df.shape[0]**0.5
        l2_dict = {
                'Vx' : 2.63748924855871,
                'Vy' : 2.6003214462464426,
                'Vz' : 2.113766985332456,
                'x' : 25391.823604180147,
                'y' : 25609.50935919694,
                'z' : 20668.126741013515,
                'delta_seconds' : 3586.29103822237}

        for c in self.l2.index:
            saved_l2 = c.replace('_sim', '')
            if saved_l2 in l2_dict.keys():
                self.l2[c] = l2_dict[saved_l2]
        
    @staticmethod
    def columns_check(columns, df_columns):
        if not columns:
            return df_columns
        return columns
        
    def z_norm(self, df, columns=None):
        columns = self.columns_check(columns, df.columns)
        return (df[columns] - self.mean[columns]) / self.std[columns]
    
    def l2_norm(self, df, columns=None):
        columns = self.columns_check(columns, df.columns)
       
        return df[columns] / self.l2[columns]
        
    def back_z_norm(self, df, columns=None):
        try:
            columns = self.columns_check(columns, df.columns)
        except:
            print("df должен быть DataFrame или columns должен быть заполнен")
            return None
        if not type(df) is pd.core.frame.DataFrame:
            df = pd.DataFrame(data=df, columns=columns)
            
    def back_l2_norm(self, df, columns=None):
        try:
            columns = self.columns_check(columns, df.columns)
        except:
            print("df должен быть DataFrame или columns должен быть заполнен")
            return None
        if not type(df) is pd.core.frame.DataFrame:
            df = pd.DataFrame(data=df, columns=columns)
            
        return df[columns] * self.l2[columns]
            

In [ ]:
def split_data(values, coeff=0.9):
    # coeff - доля спутников для тренировки
    split = int(np.floor(coeff * values))
    indices = list(range(values))
    np.random.shuffle(indices)
    train_indices, test_indices = indices[:split], indices[split:]
    return train_indices, test_indices


In [ ]:
def split_folds(indices, n_folds):
    # делит список индексов на n_folds частей
    avg = len(indices) / float(n_folds)
    result = []
    last = 0
    for _ in range(n_folds):
        result.append(indices[int(last):int(last + avg)])
        last += avg
    return result


In [ ]:
class Data_Sat(Dataset):
    def __init__(self, data, sequence_length=20):
        self.sequence_length = sequence_length
        self.data = data
        self.satellite_dict = {}
        self.split_data()

    def split_data(self):
        # разделяет данные по каждому спутнику на отдельные секвенции длиной sequence_length каждая
        # и записывает их в словарь self.satellite_dict
        # Если значений не хватило до sequence то дописывает нули

        for ind, satellite in enumerate(self.data['sat_id'].unique()):
            # берем данные по одному спутнику начиная со столбца delta_seconds (нулевой столбец sat_id пропускаем)
            sat_data = self.data.query('sat_id==@satellite').loc[:, 'delta_seconds':]
            sequence_count = int(math.ceil(sat_data.shape[0] / self.sequence_length))
            samples_sat = np.zeros((sequence_count * self.sequence_length, sat_data.shape[1]))
            samples_sat[: sat_data.shape[0]] = sat_data.values
            self.satellite_dict[ind] = samples_sat.reshape(sequence_count, self.sequence_length, -1)

    def generate_samples(self, max_sequence_count=100, last_sequence=False):
        # генерирует отдельные наборы последовательных секвенций, аугментируя данные: 
        # разбивает данные по одному спутнику (если sequence_count больше чем max_sequence_count)
        # на несколько отдельных последовательностей длиной max_sequence_count
        # например спутник с размером (sequence_count=200, sequence=20,...)
        # функция преобразует в 2 спутника размером (max_sequence_count=100, sequence=20, ...)
        # last_sequence=False - не добавляет последнюю sequence
        self.samples = []

        for sat in self.satellite_dict.values():
            sequence_count = sat.shape[0]
            if not last_sequence:
                sequence_count -= 1
            if  sequence_count > max_sequence_count:
                samples_count = math.ceil(sequence_count / max_sequence_count)
                step = (sequence_count - max_sequence_count) / (samples_count - 1)
                for sample in range(samples_count):
                    next_step = round(step * sample)
                    self.samples.append(self.data_casting(sat[next_step: next_step + max_sequence_count]))

    @staticmethod
    def data_casting(data):
        # вычитает из значений симуляции начальную ошибку.
        # начальная ошибка равна x_sym[0] - x[0] и аналогично для y, z и т.д.
        for i in range(1, 7, 1):
            data[..., i + 6] -= data[0, 0, i + 6] - data[0, 0, i]
        return data
    
    
    def predict_to_df(self, predicts):
        # Переводит predict в датафрейм и добавляет id
        self.result = pd.DataFrame()
        
        for ind, sequense in enumerate(predicts):
            filters = (np
                       .abs(self.satellite_dict[ind].reshape(sequense.shape[0], -1))
                       .sum(axis=1) != 0
                      )
            self.result = self.result.append(pd.DataFrame(sequense[filters]), ignore_index=True)
        
        assert self.result.shape[0] == self.data.shape[0]
        self.result['id'] = self.data['id'].values
        self.result = self.result[['id', 0, 1, 2, 3, 4, 5]]
        self.result.columns = ['id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']

    def __len__(self):
        """
        Returns total number of samples
        """
        return len(self.samples)

    def __getitem__(self, index):
        """
        
        :param index: 
        :return: one-satellite sample [max_sequence_count, sequence_length, gt + in values]
        """
        return FloatTensor(self.samples[index])
    

In [ ]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are point twice 
    return torch.mean(torch.abs(satellite_predicted_values - satellite_true_values) 
        / (torch.abs(satellite_predicted_values) + torch.abs(satellite_true_values)))

In [ ]:
def do_epoch(model, loss_function, data, batch_size, optimizer=None, name=None):
    """
       Генерация одной эпохи
    """
    epoch_loss = 0
   
    max_sequence_count, sequence_length = data[0].shape[0], data[0].shape[1]
    loader = torch.utils.data.DataLoader(data, batch_size=batch_size)
    batch_count = len(loader)
   
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batch_count) as progress_bar:               
            for i, sample in enumerate(loader):
                # перестановка осей. Стало - [max_sequence_count, sequence_length, batch_size, values]
                sample = sample.permute(1, 2, 0, 3)  
                model.init_hidden(sample.shape[2])
                for sequence in sample:
                    X_batch, y_batch = (sequence[..., :7]).to(device), (sequence[..., 7:]).to(device)
                    prediction = model(X_batch)
                    
                    loss = smape(prediction, y_batch)  # используем целевую метрику в качестве Loss
                    epoch_loss += loss.item()
                    
                    if is_train:
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                
                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}'.format(
                    name, loss.item())
                )
            
            epoch_loss /= (i + 1) * max_sequence_count
            score = float((1-epoch_loss) * 100)
            
            progress_bar.set_description(f'Epoch {name} -  score: {score:.2f}')

    return score

In [ ]:
def cross_validation(model, data, folds, loss_function, optimizer=None, scheduler=None,
        epochs_count=1, batch_size=1, plot_draw=False):
    """
    тренировка модели с кросс-валидацией и валидацией после каждой эпохи, валидация есть по умолчанию.
    Выводит списки fold_train_history fold_val_history.
    """
    fold_train_history = []
    fold_val_history = []
    
    optim_default_state = optimizer.state_dict() if optimizer else None
    sched_default_state = scheduler.state_dict() if scheduler else None
        
    for j, fold in enumerate(folds):

        #Возврат оптимизатора к изначальным значениям
        if optimizer:        
            optimizer.load_state_dict(optim_default_state)

        #Scheduler на каждом фолде заново определяется
        if scheduler:
            scheduler.load_state_dict(sched_default_state)

        #Сброс параметров модели на каждом фолде
        for name, module in model.named_children():
            print('resetting ', name)
            module.reset_parameters()
        
        print('Fold: ', j+1, '\n')
        
        val_data = data.loc[fold]
        val_dataset = Data_Sat(val_data, sequence_length)
        val_dataset.generate_samples(max_sequence_count=max_sequence_count,  last_sequence=False)

        train_data = data.loc[[index for nfold in folds for index in nfold if nfold != fold]]
        train_dataset = Data_Sat(train_data, sequence_length)
        train_dataset.generate_samples(max_sequence_count=max_sequence_count, last_sequence=False)

        

        train_history, val_history = fit(model, loss_function, batch_size, epoch_count, optimizer, scheduler,
                                         train_dataset, val_dataset, plot_draw)
                   
        fold_val_history.append(val_history[-1])
        fold_train_history.append(train_history[-1])
    return fold_train_history, fold_val_history

In [ ]:
def fit(model, loss_function, batch_size=1, epochs_count=10, optimizer=None,  
        scheduler=None, train_dataset=None, val_dataset=None, plot_draw=False):
    """
    Функция тренировки
    return: (list) значение Score для Train и Val на каждой эпохе
    """
    train_history = []
    val_history = []
    for epoch in range(epochs_count):
            name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
            epoch_train_score = 0
            epoch_val_score = 0
            if train_dataset:
                epoch_train_score = do_epoch(model, loss_function, train_dataset, batch_size, 
                                              optimizer, name_prefix + 'Train:')
                train_history.append(epoch_train_score)

            if val_dataset:
                name = '  Val:'
                if not train_dataset:
                    name = ' Test:'
                epoch_val_score = do_epoch(model, loss_function, val_dataset, batch_size, 
                                             optimizer=None, name=name_prefix + name)
                val_history.append(epoch_val_score)
                
                scheduler.step(epoch_val_score)
            else:
                scheduler.step(epoch_train_score)



    if plot_draw:
            draw_plot(train_history, val_history)
        
    return train_history, val_history

In [ ]:
def draw_plot(train_loss_history, val_loss_history):
    """
    Рисует lineplot
    """
    data = pd.DataFrame(data=[train_loss_history, val_loss_history], index=['Train', 'Val']).T
    plt.figure(figsize=(15, 6))
    sns.set(style='darkgrid')
    ax = sns.lineplot(data=data, markers = ["o", "o"], palette='bright')
    plt.title("Line Plot", fontsize = 20)
    plt.xlabel("Epoch", fontsize = 15)
    plt.ylabel("Loss", fontsize = 15)
    plt.show()

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim=7, output_dim=6, lstm_hidden_dim=20, 
                 lstm_layers_count=1, bidirectional=False, dropout=0,
                 previous_hid_state=True):
        super().__init__()
        
        self.previous_hid_state = previous_hid_state
        self.input_dim = input_dim 
        self.lstm_layers_count = lstm_layers_count
        self.lstm_hidden_dim = lstm_hidden_dim
            
        self.lstm = nn.LSTM(input_size = self.input_dim, 
                            hidden_size = self.lstm_hidden_dim,
                            num_layers = self.lstm_layers_count,
                            bidirectional=bidirectional,
                            bias=True,
                            dropout=dropout
                           )
        
        self.linear = nn.Linear(lstm_hidden_dim*(1+bidirectional), output_dim, bias=True)
        
    def init_hidden(self, batch_size):
        self.h = torch.zeros(self.lstm_layers_count * (2 if bidirectional else 1), 
                             batch_size, self.lstm_hidden_dim).to(device)
        self.c = torch.zeros(self.lstm_layers_count * (2 if bidirectional else 1), 
                             batch_size, self.lstm_hidden_dim).to(device)
                   

        
    def forward(self, inputs):
        if not self.previous_hid_state:
            self.init_hidden(inputs.shape[1])
        lstm_out, (self.h, self.c) = self.lstm.forward(inputs, (self.h, self.c))
        linear_out = self.linear.forward(lstm_out)
        self.h = self.h.detach()
        self.c = self.c.detach()
        
        return linear_out

In [ ]:
def predict(model, sat_data):
    """
    Получает на вход модель и разделенные на sequences_count, sequence_length данные. Предсказывает реальные значение по спутнику.
    Выводит Tensor формы (n_samples, n_features).
    """
    sequences_count, sequence_length, _ = sat_data.shape
    result = torch.zeros((sequences_count*sequence_length, 6)).to(device)
    model.eval()
    model.init_hidden(1)
    for i, seq in enumerate(sat_data):
        inputs = FloatTensor(seq[:, None, :])
        predicted = model(inputs)
        
        predicted = predicted.view(sequence_length, -1).detach()
        
        result[i*sequence_length : (i+1)*sequence_length] = predicted
        
    return result

In [ ]:
#data preparation
data = pd.read_csv(data_folder + 'train.csv', parse_dates=['epoch'])
columns = ['id', 'sat_id', 'delta_seconds', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim',
           'x', 'y', 'z', 'Vx', 'Vy', 'Vz']
data_with_dt = add_delta_time(data, columns)
data_with_dt.set_index(keys='sat_id', drop=False, inplace=True)


In [ ]:
#data normalization
normalizer = Norm(data_with_dt, ['id', 'sat_id', ])
norm_data = normalizer.l2_norm(data_with_dt)

In [ ]:
#data splitting
np.random.seed(42)

train_indices, test_indices = split_data(len(data['sat_id'].unique()))
folds = split_folds(train_indices, 5)
test_data = norm_data.loc[test_indices]

In [ ]:
# data settings
sequence_length = 5
max_sequence_count = 50

# train settings
batch_size = 10
epoch_count = 10
plot_draw = False

# optimizer settings
learning_rate = 1e-3
weight_decay = 0

# model settings
lstm_hidden_dim = 10
lstm_hidden_lauers_count = 1
bidirectional = False
dropout = 0

# scheduler settings
factor = 0.1
patience = 2
threshold = 1e-2

model = LSTM(lstm_hidden_dim=lstm_hidden_dim,
             lstm_layers_count=lstm_hidden_lauers_count,
             bidirectional=bidirectional,
             dropout=dropout,
            ).to(device)

loss_function = torch.nn.MSELoss()
optimizer = optim.Adam(
                        model.parameters(),
                        lr=learning_rate, 
                        weight_decay=weight_decay
                    )

scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=factor, 
                              patience=patience, verbose=True, threshold=threshold
                              )

In [ ]:
file_name = 'score_80_v1.model'
path = data_folder  + file_name

In [ ]:
save_model(path, model, optimizer, scheduler, train_hist, val_hist)

In [ ]:
load_model(path, model, optimizer, scheduler, _, _)

In [ ]:
tr_hist, val_hist = cross_validation(model, norm_data, folds, loss_function, optimizer, scheduler, epochs_count=epoch_count,
    batch_size=batch_size, plot_draw=plot_draw
   )

In [ ]:
print('Mean_train_score: ', np.mean(tr_hist), ' Mean_val_score: ', np.mean(val_hist))

In [ ]:
train_dataset = Data_Sat(norm_data, sequence_length)
train_dataset.generate_samples(max_sequence_count, False)

In [ ]:
# Train model
train_hist, val_hist = fit(model, loss_function, batch_size=batch_size, epochs_count=epoch_count, optimizer=optimizer,  
        scheduler=scheduler, train_dataset=train_dataset, val_dataset=None, plot_draw=False)

In [ ]:
test_dataset = Data_Sat(test_data, sequence_length)

In [ ]:
#Predict test and compute score
metric = 0
test_predicts = []
for sat in test_dataset.satellite_dict:
    sat_data = test_dataset.satellite_dict[sat]
    X = FloatTensor(sat_data[..., :7]).to(device)
    y = FloatTensor(sat_data[..., 7:]).view(-1, 6).to(device)
    predicts = predict(model, X)[y!=0].view(-1, 6)
    test_predicts.append(predicts.cpu().detach().numpy())
    metric += smape(predicts, 
                    y[y!=0].view(-1, 6)
                   )
    
metric /= len(test_dataset.satellite_dict)
score = (1-metric)*100
print(f'Test score: {float(score.cpu()):.2f}')

In [ ]:
def submission():
    test = pd.read_csv(data_folder + "/test.csv", parse_dates=['epoch'])
    columns = ['id', 'sat_id', 'delta_seconds', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
    test_with_dt = add_delta_time(test, columns)
    normalizer = Norm(test_with_dt, ignore_column=['id', 'sat_id'])
    norm_test = normalizer.l2_norm(test_with_dt)
    test_dataset = Data_Sat(norm_test, 5)
    test_predicts = []
    for sat in test_dataset.satellite_dict:
        sat_data = test_dataset.satellite_dict[sat]
        X = FloatTensor(sat_data).to(device)
        predicts = predict(model, X)
        test_predicts.append(predicts.cpu().detach().numpy())
    test_dataset.predict_to_df(test_predicts)
    submis = test_dataset.result * normalizer.l2.drop(['sat_id','delta_seconds']).values
    submis['id'] = submis['id'].astype('int')
    submis.to_csv(data_folder + "/submission_80score.csv", index=False)
    
submission()